Conteúdo desse notebook retirado de: https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/

In [1]:
import openai
from openai import OpenAI
import os

chave = 'sua-chave'
client = OpenAI(
   api_key=chave,
)

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [3]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature  # the maximum number of tokens the model can output
    )
    return response.choices[0].message.content

In [4]:
user_input = "tell me about your laptops"

In [5]:
# Step 1: Check input to see if it flags the Moderation API or is a prompt injection
response = client.moderations.create(input=user_input)
moderation_output = response.results[0]

if moderation_output.flagged:
    print("Step 1: Input flagged by Moderation API.")
    print("Sorry, we cannot process this request.")

else: print("Step 1: Input passed moderation check.")

Step 1: Input passed moderation check.


In [6]:
products_file = 'products.json'
import json

from collections import defaultdict

def get_products():
    with open(products_file, 'r') as file:
        products = json.load(file)
    return products

def get_products_and_category():
    """
    Used in L5
    """
    products = get_products()
    products_by_category = defaultdict(list)
    for product_name, product_info in products.items():
        category = product_info.get('category')
        if category:
            products_by_category[category].append(product_info.get('name'))
    
    return dict(products_by_category)

In [7]:
get_products()

{'TechPro Ultrabook': {'name': 'TechPro Ultrabook',
  'category': 'Computers and Laptops',
  'brand': 'TechPro',
  'model_number': 'TP-UB100',
  'warranty': '1 year',
  'rating': 4.5,
  'features': ['13.3-inch display',
   '8GB RAM',
   '256GB SSD',
   'Intel Core i5 processor'],
  'description': 'A sleek and lightweight ultrabook for everyday use.',
  'price': 799.99},
 'BlueWave Gaming Laptop': {'name': 'BlueWave Gaming Laptop',
  'category': 'Computers and Laptops',
  'brand': 'BlueWave',
  'model_number': 'BW-GL200',
  'warranty': '2 years',
  'rating': 4.7,
  'features': ['15.6-inch display',
   '16GB RAM',
   '512GB SSD',
   'NVIDIA GeForce RTX 3060'],
  'description': 'A high-performance gaming laptop for an immersive experience.',
  'price': 1199.99},
 'PowerLite Convertible': {'name': 'PowerLite Convertible',
  'category': 'Computers and Laptops',
  'brand': 'PowerLite',
  'model_number': 'PL-CV300',
  'warranty': '1 year',
  'rating': 4.3,
  'features': ['14-inch touchscreen'

In [8]:
def find_category_and_product_only(user_input):
    delimiter = "####"
    system_message = f"""
    You will be provided with customer service queries. \
    The customer service query will be delimited with \
    {delimiter} characters.
    Output a python list of objects, where each object has \
    the following format:
        'category': <one of Computers and Laptops, \
        Smartphones and Accessories, \
        Televisions and Home Theater Systems, \
        Gaming Consoles and Accessories, 
        Audio Equipment, Cameras and Camcorders>,
    OR
        'products': <a list of products that must \
        be found in the allowed products below>
    
    Where the categories and products must be found in \
    the customer service query.
    If a product is mentioned, it must be associated with \
    the correct category in the allowed products list below.
    If no products or categories are found, output an \
    empty list.
    
    Allowed products: 
    
    Computers and Laptops category:
    TechPro Ultrabook
    BlueWave Gaming Laptop
    PowerLite Convertible
    TechPro Desktop
    BlueWave Chromebook
    
    Smartphones and Accessories category:
    SmartX ProPhone
    MobiTech PowerCase
    SmartX MiniPhone
    MobiTech Wireless Charger
    SmartX EarBuds
    
    Televisions and Home Theater Systems category:
    CineView 4K TV
    SoundMax Home Theater
    CineView 8K TV
    SoundMax Soundbar
    CineView OLED TV
    
    Gaming Consoles and Accessories category:
    GameSphere X
    ProGamer Controller
    GameSphere Y
    ProGamer Racing Wheel
    GameSphere VR Headset
    
    Audio Equipment category:
    AudioPhonic Noise-Canceling Headphones
    WaveSound Bluetooth Speaker
    AudioPhonic True Wireless Earbuds
    WaveSound Soundbar
    AudioPhonic Turntable
    
    Cameras and Camcorders category:
    FotoSnap DSLR Camera
    ActionCam 4K
    FotoSnap Mirrorless Camera
    ZoomMaster Camcorder
    FotoSnap Instant Camera
    
    Only output the list of objects, with nothing else.
    """
    user_message_1 = f"""
     tell me about the smartx pro phone and \
     the fotosnap camera, the dslr one. \
     Also tell me about your tvs """
    messages =  [  
    {'role':'system', 
     'content': system_message},    
    {'role':'user', 
     'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    category_and_product_response_1 = get_completion_from_messages(messages)
    return category_and_product_response_1

category_and_product_response = find_category_and_product_only(user_input)

In [9]:
category_and_product_response

"[{'category': 'Computers and Laptops'}]"

In [10]:
def read_string_to_list(input_string):
    if input_string is None:
        return None

    try:
        input_string = input_string.replace("'", "\"")  # Replace single quotes with double quotes for valid JSON
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None

In [11]:
category_and_product_list = read_string_to_list(category_and_product_response)

In [12]:
category_and_product_list

[{'category': 'Computers and Laptops'}]

In [13]:


# product look up (either by category or by product within category)
def get_product_by_name(name):
    products = get_products()
    return products.get(name, None)

def get_products_by_category(category):
    products = get_products()
    return [product for product in products.values() if product["category"] == category]

In [14]:
def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string

In [15]:
product_information = generate_output_string(category_and_product_list)

In [16]:
print(product_information)

{
    "name": "TechPro Ultrabook",
    "category": "Computers and Laptops",
    "brand": "TechPro",
    "model_number": "TP-UB100",
    "warranty": "1 year",
    "rating": 4.5,
    "features": [
        "13.3-inch display",
        "8GB RAM",
        "256GB SSD",
        "Intel Core i5 processor"
    ],
    "description": "A sleek and lightweight ultrabook for everyday use.",
    "price": 799.99
}
{
    "name": "BlueWave Gaming Laptop",
    "category": "Computers and Laptops",
    "brand": "BlueWave",
    "model_number": "BW-GL200",
    "warranty": "2 years",
    "rating": 4.7,
    "features": [
        "15.6-inch display",
        "16GB RAM",
        "512GB SSD",
        "NVIDIA GeForce RTX 3060"
    ],
    "description": "A high-performance gaming laptop for an immersive experience.",
    "price": 1199.99
}
{
    "name": "PowerLite Convertible",
    "category": "Computers and Laptops",
    "brand": "PowerLite",
    "model_number": "PL-CV300",
    "warranty": "1 year",
    "rating": 4

In [17]:
# Step 4: Answer the user question
delimiter = "####"
system_message = f"""
You are a customer service assistant for a large electronic store. \
Respond in a friendly and helpful tone, with concise answers. \
Make sure to ask the user relevant follow-up questions.
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
    {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
]
#all_messages + messages
final_response = get_completion_from_messages(messages)

In [18]:
print(final_response)

We have a variety of laptops available, including the TechPro Ultrabook, BlueWave Gaming Laptop, PowerLite Convertible, and more. Each laptop has different features and price points to suit your needs. Is there a specific type of laptop you are looking for, such as for gaming, work, or everyday use? Let me know if you need more details on any specific model!


In [19]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    response = client.moderations.create(input=user_input)
    moderation_output = response.results[0]

    if moderation_output.flagged:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    
    category_and_product_response = find_category_and_product_only(user_input)
    #print(print(category_and_product_response)
    # Step 2: Extract the list of products
    category_and_product_list = read_string_to_list(category_and_product_response)
    #print(category_and_product_list)

    # Step 3: If products are found, look them up
    product_information = generate_output_string(category_and_product_list)

    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug: print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    return final_response, all_messages
    

user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response, _ = process_user_message(user_input, [])
print(response)

Error: Invalid JSON string
Step 4: Generated response to user question.
- The SmartX Pro phone is a high-end smartphone with a powerful processor, advanced camera features, and a sleek design. It offers a great user experience with its fast performance and vibrant display. Do you have any specific questions about its features or specifications?

- The FotoSnap camera is a DSLR camera known for its professional-grade image quality and versatility. It offers manual controls for advanced photographers and produces stunning photos with its high-resolution sensor. Is there anything specific you would like to know about its features or capabilities?

- Our range of TVs includes various sizes and features such as 4K resolution, smart TV capabilities, and HDR technology for enhanced picture quality. Do you have a preference for a specific brand or size when it comes to choosing a TV?


In [24]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))
 
    return pn.Column(*panels)

In [27]:
# OBS.: Essa parte aqui não funciona bem no vscode por algum motivo. 
# Caso queira fazer o chat bonitinho, use o jupyter-notebook
# É só ir em uma célula vazia aqui e digitar !jupyter-notebook
# E ai abrir o seu arquivo e executar tudo de novo com shift+enter na nova interface.

import panel as pn  # GUI
pn.extension()
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=1000),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'a364286c-3c14-48f4-91f6-732a8e645a3d': {'version…

In [22]:
# Alternativa: Fazer site com flask